In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

In [2]:
!pip install rouge

In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9cfdf0f4bca8fb368737d040dac5c3dad604e5be05540ecee82ddcefd84e30f5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [4]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing in

In [5]:
import json
import argparse
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import sys
import math
import warnings
import evaluate
from torch.optim import AdamW

metric = evaluate.load("rouge")
warnings.filterwarnings("ignore")

# Transformer imports
from transformers import (
    RobertaTokenizer, 
    RobertaForSequenceClassification, 
    BartTokenizer, 
    BartForConditionalGeneration,
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    T5Tokenizer, 
    T5ForConditionalGeneration,
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq,
    # AdamW, 
    get_linear_schedule_with_warmup,
    BertTokenizer,
    BertModel
)
from transformers.file_utils import PushToHubMixin

# PEFT imports
from peft import (
    get_peft_config, 
    get_peft_model, 
    get_peft_model_state_dict, 
    PrefixTuningConfig, 
    TaskType,
    PeftModel
)

# Evaluation imports
# from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from scipy.spatial.distance import cosine
# from rouge import Rouge
from bert_score import score as bert_score

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Define paths for Kaggle
BASE_PATH = '/kaggle/input/nlp-project-dataset/puma_dataset'
OUTPUT_PATH = '/kaggle/working/'

# Create directories
# os.makedirs(f"{OUTPUT_PATH}/generated", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/checkpoints", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/", exist_ok=True)

# Download NLTK data
import nltk
nltk.download('punkt')

2025-04-15 17:35:48.085506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744738548.276294      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744738548.327094      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Classifier DataLoader Implementation
class ClassifierCustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {"EXPERIENCE": 0, "SUGGESTION": 1, "INFORMATION": 2, "CAUSE": 3, "QUESTION": 4}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Extract the correct input text and label based on JSON structure
        # Assuming each example has a 'labelled_answer_spans' and 'labelled_summaries' field
        # with keys for each perspective type
        
        # Get all available perspective keys
        perspectives = list(item['labelled_summaries'].keys())
        
        # Select the first perspective as the label (can be modified to handle multiple perspectives)
        if perspectives:
            chosen_perspective = perspectives[0]
            label = chosen_perspective
            input_text = item['labelled_answer_spans'].get(chosen_perspective, "")
        else:
            # Fallback if no perspectives are found
            label = "INFORMATION"  # Default label
            input_text = ""
            
        # Encode label to integer
        label_encoded = self.label_map.get(label, 0)  # Default to 0 if label not in map
        
        inputs = self.tokenizer(
            input_text,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        )
        
        # Return input_ids, attention_mask, and label
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "label": torch.tensor(label_encoded)  # Return integer label directly
        }

# Seq2Seq DataLoader Implementation
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
      
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Extract answers from the JSON structure
        answers = item.get('answers', [])
        non_empty_sentences = ' '.join([ans.replace('\n', '') for ans in answers if ans])
        
        # Extract perspective information
        # We'll select the first perspective available in labelled_summaries
        perspectives = list(item.get('labelled_summaries', {}).keys())
        if not perspectives:
            # Fallback to default perspective if none found
            perspective = "INFORMATION"
            summary = ""
        else:
            perspective = perspectives[0]
            # Use the first summary associated with this perspective
            summary = item['labelled_summaries'][perspective]
        
        # Define perspective attributes based on the perspective type
        defn = ""
        start_with = ""
        tone_attribute = ""
        
        if perspective == "SUGGESTION":
            defn = "Defined as advice or recommendations to assist users in making informed medical decisions, solving problems, or improving health issues."
            start_with = "It is suggested"
            tone_attribute = "Advisory, Recommending"
            
        elif perspective == "INFORMATION":
            defn = "Defined as knowledge about diseases, disorders, and health-related facts, providing insights into symptoms and diagnosis."
            start_with = "For information purposes"
            tone_attribute = "Informative, Educational"
            
        elif perspective == "EXPERIENCE":
            defn = "Defined as individual experiences, anecdotes, or firsthand insights related to health, medical treatments, medication usage, and coping strategies"
            start_with = "In user's experience"
            tone_attribute = "Personal, Narrative"
            
        elif perspective == "CAUSE":
            defn = "Defined as reasons responsible for the occurrence of a particular medical condition, symptom, or disease"
            start_with = "Some of the causes"
            tone_attribute = "Explanatory, Causal"
            
        elif perspective == "QUESTION":
            defn = "Defined as inquiry made for deeper understanding."
            start_with = "It is inquired"
            tone_attribute = "Seeking Understanding"
        
        # Check if the summary already starts with the appropriate phrase
        if summary and len(set(summary.split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
            target_text = summary
        else:
            target_text = start_with + " " + (summary or "")
        
        # Create the task prefix for the model
        task_prefix = (f"Adhering to the condition of 'begin summary with' and 'tone of summary' and "
                       f"summarize according to {perspective} and start the summary with '{start_with.strip()}'. "
                       f"Maintain summary tone as {tone_attribute.strip()}. Definition of perspective: {defn.strip().lower()} "
                       f"Content to summarize: {non_empty_sentences} Question: {item.get('question', '').strip()}.")
        
        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length, truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
            
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
            "perspective": perspective,
            "Summary": summary
        }

# DataLoader Creation Functions
def create_dataloader(train_dataset, valid_dataset, VALID_BATCH_SIZE, TRAIN_BATCH_SIZE):
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=True)
    return train_dataloader, valid_dataloader

def test_create_dataloader(test_dataset, TEST_BATCH_SIZE):
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)
    return test_dataloader

In [7]:
# Validation function for classifier
def classifier_validation(valid_dataloader, model, VALID_BATCH_SIZE, optimizer, scheduler):
    print("Validation processing...")
    model.eval()    
    valid_losses = []
  
    with torch.no_grad():
        for i, batch in enumerate(tqdm(valid_dataloader)):
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
         
            loss = output.loss
            
            valid_losses.append(loss.item())  
        
    valid_loss = np.mean(valid_losses) if len(valid_losses) > 0 else 0.0
    return valid_loss

# Function to train the classifier model
def train_classifier(train_data_path, valid_data_path, output_path, 
                    train_batch_size=8, valid_batch_size=4, 
                    learning_rate=1e-5, warmup_steps=4000, epochs=5):
    
    # Load data
    with open(train_data_path, 'r') as json_file:
        train_data = json.load(json_file)

    with open(valid_data_path, 'r') as json_file:
        valid_data = json.load(json_file)
    
    best_loss = float('inf')
    print("Initial best_loss:", best_loss)
    last_epoch = 0

    # Initialize model and tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
    model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)
            
    # Create datasets and dataloaders
    train_dataset = ClassifierCustomDataset(train_data, tokenizer)
    eval_dataset = ClassifierCustomDataset(valid_data, tokenizer)
    train_dataloader, eval_dataloader = create_dataloader(train_dataset, eval_dataset, valid_batch_size, train_batch_size)
    
    # Define optimizer and learning rate scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(train_dataloader) * epochs)

    start_epoch = last_epoch + 1
    num_batches = len(train_dataloader)

    # Move model to device
    model.to(device)
    
    # Training loop
    for epoch in range(start_epoch, start_epoch + epochs):
        model.train()
        print(f"{'#'*50} Epoch: {epoch} {'#'*50}")
        train_losses = []
        
        for i, batch in enumerate(tqdm(train_dataloader)):
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_losses.append(loss.detach())
         
        train_losses = [loss.item() for loss in train_losses] 
        train_loss = np.mean(train_losses)
        print(f"Train loss: {train_loss} for epoch: {epoch}")
        
        # Validation
        valid_loss = classifier_validation(eval_dataloader, model, valid_batch_size, optimizer, scheduler)
        print(f"Validation loss: {valid_loss} for epoch: {epoch}")
        
        # Save best checkpoint
        if valid_loss < best_loss:
            best_loss = valid_loss
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optimizer.state_dict(),
                'sched_state_dict': scheduler.state_dict(),
                'loss': best_loss,
                'epoch': epoch
            }
           
            torch.save(state_dict, f"{output_path}/best_ckpt_epoch={epoch}.ckpt")
            print(round(best_loss,5))
            print("*"*10 + "Current best checkpoint is saved." + "*"*10)
    
    return model, tokenizer

In [8]:
class EvaluationMetrics:
    def __init__(self, predictions, references):
        self.predictions = predictions
        self.references = references

    def compute_rouge_score(self):
        rouge = Rouge()
        rouge_l_f1, rouge_l_recall, rouge_l_precision = [], [], []
        rouge_1_f1, rouge_1_recall, rouge_1_precision = [], [], []
        rouge_2_f1, rouge_2_recall, rouge_2_precision = [], [], []
        for prediction, reference in zip(self.predictions, self.references):
            scores = rouge.get_scores(prediction, reference)[0]
            
            rouge_l_f1.append(scores["rouge-l"]["f"])
            rouge_l_recall.append(scores["rouge-l"]["r"])
            rouge_l_precision.append(scores["rouge-l"]["p"])
            
            rouge_1_f1.append(scores["rouge-1"]["f"])
            rouge_1_recall.append(scores["rouge-1"]["r"])
            rouge_1_precision.append(scores["rouge-1"]["p"])
            
            rouge_2_f1.append(scores["rouge-2"]["f"])
            rouge_2_recall.append(scores["rouge-2"]["r"])
            rouge_2_precision.append(scores["rouge-2"]["p"])

        results = {
            "rouge_l": {
                "f1": np.mean(rouge_l_f1) * 100,
                "recall": np.mean(rouge_l_recall) * 100,
                "precision": np.mean(rouge_l_precision) * 100
            },
            "rouge_1": {
                "f1": np.mean(rouge_1_f1) * 100,
                "recall": np.mean(rouge_1_recall) * 100,
                "precision": np.mean(rouge_1_precision) * 100
            },
            "rouge_2": {
                "f1": np.mean(rouge_2_f1) * 100,
                "recall": np.mean(rouge_2_recall) * 100,
                "precision": np.mean(rouge_2_precision) * 100
            }
        }
        
        return results

    def compute_meteor_score(self):
        meteor = load_metric('meteor')
        scores = []
        for prediction, reference in zip(self.predictions, self.references):
            score = meteor.compute(predictions=[prediction], references=[reference])
            scores.append(score["meteor"])

        average_meteor_score = np.mean(scores)
        
        return {"meteor": average_meteor_score}
    
    def compute_bleu_scores(self):
        bleu_1, bleu_2, bleu_3, bleu_4 = [], [], [], []
        smoothie = SmoothingFunction().method4
        for prediction, reference in zip(self.predictions, self.references):
            reference = [reference.split()]  # BLEU score API requires tokenized sentences
            prediction = prediction.split()
            
            bleu_1.append(sentence_bleu(reference, prediction, weights=(1, 0, 0, 0), smoothing_function=smoothie))
            bleu_2.append(sentence_bleu(reference, prediction, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie))
            bleu_3.append(sentence_bleu(reference, prediction, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie))
            bleu_4.append(sentence_bleu(reference, prediction, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie))

        results = {
            "bleu_1": np.mean(bleu_1),
            "bleu_2": np.mean(bleu_2),
            "bleu_3": np.mean(bleu_3),
            "bleu_4": np.mean(bleu_4)
        }
        
        return results
    
    def compute_bertscore(self, lang="en"):
        P, R, F1 = bert_score(self.predictions, self.references, lang=lang)
        return {
            "bertscore": {
                "precision": P.mean().item(),
                "recall": R.mean().item(),
                "f1": F1.mean().item()
            }
        }
    
    def evaluate_all(self):
        """Run all evaluation metrics and return a combined dictionary of results"""
        results = {}
        
        # Get all metrics
        rouge_results = self.compute_rouge_score()
        meteor_results = self.compute_meteor_score()
        bleu_results = self.compute_bleu_scores()
        bert_results = self.compute_bertscore()
        
        # Combine results
        results.update(rouge_results)
        results.update(meteor_results)
        results.update(bleu_results)
        results.update(bert_results)
        
        return results

In [9]:
# Define paths to your data
train_data_path = '/kaggle/input/nlp-project-dataset/puma_dataset/train.json'
valid_data_path = '/kaggle/input/nlp-project-dataset/puma_dataset/valid.json'
output_path = '/kaggle/working/'

# Train classifier
classifier_model, classifier_tokenizer = train_classifier(
    train_data_path=train_data_path,
    valid_data_path=valid_data_path,
    output_path=output_path,
    train_batch_size=8,
    valid_batch_size=4,
    learning_rate=1e-5,
    warmup_steps=4000,
    epochs=5
)

Initial best_loss: inf


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Validation loss: 0.002871677746103766 for epoch: 4
0.00287
**********Current best checkpoint is saved.**********
################################################## Epoch: 5 ##################################################


100%|██████████| 280/280 [02:41<00:00,  1.73it/s]


Train loss: 0.00783991087976444 for epoch: 5
Validation processing...


100%|██████████| 240/240 [00:14<00:00, 16.33it/s]


Validation loss: 0.0017476299544796348 for epoch: 5
0.00175
**********Current best checkpoint is saved.**********


In [14]:
import json
import argparse
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import sys
import math
import warnings
import evaluate
from torch.optim import AdamW
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import time
from datetime import datetime

metric = evaluate.load("rouge")
warnings.filterwarnings("ignore")
# Transformer imports
from transformers import (
    RobertaTokenizer, 
    RobertaForSequenceClassification, 
    BartTokenizer, 
    BartForConditionalGeneration,
    AutoModelForSeq2SeqLM, 
    AutoTokenizer, 
    T5Tokenizer, 
    T5ForConditionalGeneration,
    Seq2SeqTrainer, 
    Seq2SeqTrainingArguments, 
    DataCollatorForSeq2Seq,
    get_linear_schedule_with_warmup,
    BertTokenizer,
    BertModel
)
from transformers.file_utils import PushToHubMixin
# PEFT imports
from peft import (
    get_peft_config, 
    get_peft_model, 
    get_peft_model_state_dict, 
    PrefixTuningConfig, 
    TaskType,
    PeftModel
)
# Evaluation imports
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from scipy.spatial.distance import cosine
from bert_score import score as bert_score
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
# Define paths for Kaggle
BASE_PATH = '/kaggle/input/nlp-project-dataset/puma_dataset'
OUTPUT_PATH = '/kaggle/working/'
# Create directories
# os.makedirs(f"{OUTPUT_PATH}/generated", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/checkpoints", exist_ok=True)
# os.makedirs(f"{OUTPUT_PATH}/plots", exist_ok=True)
# Download NLTK data
import nltk
nltk.download('punkt')

# Classifier DataLoader Implementation
class ClassifierCustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {"EXPERIENCE": 0, "SUGGESTION": 1, "INFORMATION": 2, "CAUSE": 3, "QUESTION": 4}
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Extract the correct input text and label based on JSON structure
        # Assuming each example has a 'labelled_answer_spans' and 'labelled_summaries' field
        # with keys for each perspective type
        
        # Get all available perspective keys
        perspectives = list(item['labelled_summaries'].keys())
        
        # Select the first perspective as the label (can be modified to handle multiple perspectives)
        if perspectives:
            chosen_perspective = perspectives[0]
            label = chosen_perspective
            input_text = item['labelled_answer_spans'].get(chosen_perspective, "")
        else:
            # Fallback if no perspectives are found
            label = "INFORMATION"  # Default label
            input_text = ""
            
        # Encode label to integer
        label_encoded = self.label_map.get(label, 0)  # Default to 0 if label not in map
        
        inputs = self.tokenizer(
            input_text,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        )
        
        # Return input_ids, attention_mask, and label
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "label": torch.tensor(label_encoded)  # Return integer label directly
        }

# Seq2Seq DataLoader Implementation
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
      
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Extract answers from the JSON structure
        answers = item.get('answers', [])
        non_empty_sentences = ' '.join([ans.replace('\n', '') for ans in answers if ans])
        
        # Extract perspective information
        # We'll select the first perspective available in labelled_summaries
        perspectives = list(item.get('labelled_summaries', {}).keys())
        if not perspectives:
            # Fallback to default perspective if none found
            perspective = "INFORMATION"
            summary = ""
        else:
            perspective = perspectives[0]
            # Use the first summary associated with this perspective
            summary = item['labelled_summaries'][perspective]
        
        # Define perspective attributes based on the perspective type
        defn = ""
        start_with = ""
        tone_attribute = ""
        
        if perspective == "SUGGESTION":
            defn = "Defined as advice or recommendations to assist users in making informed medical decisions, solving problems, or improving health issues."
            start_with = "It is suggested"
            tone_attribute = "Advisory, Recommending"
            
        elif perspective == "INFORMATION":
            defn = "Defined as knowledge about diseases, disorders, and health-related facts, providing insights into symptoms and diagnosis."
            start_with = "For information purposes"
            tone_attribute = "Informative, Educational"
            
        elif perspective == "EXPERIENCE":
            defn = "Defined as individual experiences, anecdotes, or firsthand insights related to health, medical treatments, medication usage, and coping strategies"
            start_with = "In user's experience"
            tone_attribute = "Personal, Narrative"
            
        elif perspective == "CAUSE":
            defn = "Defined as reasons responsible for the occurrence of a particular medical condition, symptom, or disease"
            start_with = "Some of the causes"
            tone_attribute = "Explanatory, Causal"
            
        elif perspective == "QUESTION":
            defn = "Defined as inquiry made for deeper understanding."
            start_with = "It is inquired"
            tone_attribute = "Seeking Understanding"
        
        # Check if the summary already starts with the appropriate phrase
        if summary and len(set(summary.split(" ")[:5]).intersection(set(start_with.split()))) >= 2:
            target_text = summary
        else:
            target_text = start_with + " " + (summary or "")
        
        # Create the task prefix for the model
        task_prefix = (f"Adhering to the condition of 'begin summary with' and 'tone of summary' and "
                       f"summarize according to {perspective} and start the summary with '{start_with.strip()}'. "
                       f"Maintain summary tone as {tone_attribute.strip()}. Definition of perspective: {defn.strip().lower()} "
                       f"Content to summarize: {non_empty_sentences} Question: {item.get('question', '').strip()}.")
        
        inputs = self.tokenizer(task_prefix, padding="max_length", max_length=self.max_length, truncation=True, return_tensors="pt")
        labels = self.tokenizer(target_text, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
            
        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
            "perspective": perspective,
            "Summary": summary
        }

# DataLoader Creation Functions
def create_dataloader(train_dataset, valid_dataset, VALID_BATCH_SIZE, TRAIN_BATCH_SIZE):
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=True)
    return train_dataloader, valid_dataloader

def test_create_dataloader(test_dataset, TEST_BATCH_SIZE):
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False)
    return test_dataloader

# Validation function for classifier
def classifier_validation(valid_dataloader, model, VALID_BATCH_SIZE, optimizer, scheduler):
    print("Validation processing...")
    model.eval()    
    valid_losses = []
    all_predictions = []
    all_labels = []
  
    with torch.no_grad():
        for i, batch in enumerate(tqdm(valid_dataloader)):
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            
            loss = output.loss
            valid_losses.append(loss.item())
            
            # Get predictions
            logits = output.logits
            predictions = torch.argmax(logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
    valid_loss = np.mean(valid_losses) if len(valid_losses) > 0 else 0.0
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='weighted')
    
    metrics = {
        "loss": valid_loss,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    
    return valid_loss, metrics, all_predictions, all_labels

# Function to visualize training progress
def plot_training_progress(train_losses, valid_losses, metrics_history, output_path):
    """
    Plot training and validation losses along with other metrics.
    """
    # Convert epochs to list for x-axis
    epochs = list(range(1, len(train_losses) + 1))
    
    # Plot losses
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, valid_losses, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Plot accuracy
    plt.subplot(2, 2, 2)
    plt.plot(epochs, [m['accuracy'] for m in metrics_history], 'g-')
    plt.title('Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.grid(True)
    
    # Plot F1 Score
    plt.subplot(2, 2, 3)
    plt.plot(epochs, [m['f1'] for m in metrics_history], 'c-')
    plt.title('Validation F1 Score')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.grid(True)
    
    # Plot Precision and Recall
    plt.subplot(2, 2, 4)
    plt.plot(epochs, [m['precision'] for m in metrics_history], 'm-', label='Precision')
    plt.plot(epochs, [m['recall'] for m in metrics_history], 'y-', label='Recall')
    plt.title('Precision and Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f"{output_path}/training_progress.png")
    plt.close()

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, output_path, label_map=None):
    """
    Plot confusion matrix for classifier predictions.
    """
    # Create mapping from index to label name
    if label_map:
        index_to_label = {v: k for k, v in label_map.items()}
        label_names = [index_to_label[i] for i in range(len(label_map))]
    else:
        label_names = list(range(5))  # Default to numeric labels
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                xticklabels=label_names, 
                yticklabels=label_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.savefig(f"{output_path}/confusion_matrix.png")
    plt.close()
    
    # Calculate and return class-wise metrics
    report = classification_report(y_true, y_pred, target_names=label_names, output_dict=True)
    return report

# Plot learning rate
def plot_learning_rate(learning_rates, output_path):
    """
    Plot learning rate changes over time.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(learning_rates, 'b-')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Steps')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.savefig(f"{output_path}/learning_rate.png")
    plt.close()

# Evaluate model on test data and generate predictions
def evaluate_classifier_test(test_dataloader, model, tokenizer):
    """
    Evaluate the model on test data and return predictions with analysis.
    """
    model.eval()
    all_predictions = []
    all_input_texts = []
    label_map = {"EXPERIENCE": 0, "SUGGESTION": 1, "INFORMATION": 2, "CAUSE": 3, "QUESTION": 4}
    idx_to_label = {v: k for k, v in label_map.items()}
    
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            # Decode input text for analysis
            for i in range(len(input_ids)):
                input_text = tokenizer.decode(input_ids[i], skip_special_tokens=True)
                all_input_texts.append(input_text)
                all_predictions.append(idx_to_label[predictions[i].item()])
    
    results = {
        "texts": all_input_texts,
        "predictions": all_predictions
    }
    
    return results

# Sample prediction visualization
def visualize_sample_predictions(results, num_samples=5, output_path=None):
    """
    Visualize sample predictions with text for qualitative analysis.
    """
    samples = min(num_samples, len(results['texts']))
    
    plt.figure(figsize=(12, samples * 2))
    
    for i in range(samples):
        idx = np.random.randint(0, len(results['texts']))
        text = results['texts'][idx]
        prediction = results['predictions'][idx]
        
        # Truncate text if too long
        if len(text) > 100:
            text = text[:97] + "..."
        
        plt.subplot(samples, 1, i+1)
        plt.barh([prediction], [1], color='skyblue')
        plt.title(f"Sample {i+1}: Predicted as {prediction}")
        plt.yticks([prediction])
        plt.figtext(0.5, (i+0.5)/samples - 0.05, f"Text: {text}", 
                   wrap=True, horizontalalignment='center', fontsize=8)
    
    plt.tight_layout()
    if output_path:
        plt.savefig(f"{output_path}/sample_predictions.png")
    plt.close()
    
    return

# Function to analyze distribution of predictions
def plot_prediction_distribution(predictions, output_path):
    """
    Plot distribution of predicted classes.
    """
    unique_labels = sorted(set(predictions))
    counts = [predictions.count(label) for label in unique_labels]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(unique_labels, counts)
    
    # Add count labels on top of bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', fontweight='bold')
    
    plt.title('Distribution of Predicted Classes')
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.savefig(f"{output_path}/prediction_distribution.png")
    plt.close()

# Function to train the classifier model with enhanced monitoring and visualization
def train_classifier(train_data_path, valid_data_path, output_path, 
                    train_batch_size=8, valid_batch_size=4, 
                    learning_rate=1e-5, warmup_steps=4000, epochs=5):
    
    # Load data
    with open(train_data_path, 'r') as json_file:
        train_data = json.load(json_file)
    with open(valid_data_path, 'r') as json_file:
        valid_data = json.load(json_file)
    
    best_loss = float('inf')
    print("Initial best_loss:", best_loss)
    last_epoch = 0
    # Initialize model and tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
    model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)
            
    # Create datasets and dataloaders
    train_dataset = ClassifierCustomDataset(train_data, tokenizer)
    eval_dataset = ClassifierCustomDataset(valid_data, tokenizer)
    train_dataloader, eval_dataloader = create_dataloader(train_dataset, eval_dataset, valid_batch_size, train_batch_size)
    
    # Define optimizer and learning rate scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=len(train_dataloader) * epochs)
    start_epoch = last_epoch + 1
    num_batches = len(train_dataloader)
    
    # Move model to device
    model.to(device)
    
    # Initialize tracking metrics
    train_losses_history = []
    valid_losses_history = []
    metrics_history = []
    learning_rates = []
    
    start_time = time.time()
    
    # Training loop
    for epoch in range(start_epoch, start_epoch + epochs):
        model.train()
        print(f"{'#'*50} Epoch: {epoch} {'#'*50}")
        train_losses = []
        
        # Training
        for i, batch in enumerate(tqdm(train_dataloader)):
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            
            # Track learning rate
            learning_rates.append(scheduler.get_last_lr()[0])
            
            scheduler.step()
            train_losses.append(loss.detach().item())
            
            # Log progress periodically
            if i % 50 == 0:
                print(f"Epoch {epoch}, Batch {i}/{num_batches}, Loss: {loss.item():.4f}")
         
        avg_train_loss = np.mean(train_losses)
        train_losses_history.append(avg_train_loss)
        print(f"Train loss: {avg_train_loss} for epoch: {epoch}")
        
        # Validation
        valid_loss, metrics, all_predictions, all_labels = classifier_validation(
            eval_dataloader, model, valid_batch_size, optimizer, scheduler
        )
        valid_losses_history.append(valid_loss)
        metrics_history.append(metrics)
        
        print(f"Validation metrics for epoch {epoch}:")
        for k, v in metrics.items():
            print(f"  {k}: {v:.4f}")
        
        # Save best checkpoint
        if valid_loss < best_loss:
            best_loss = valid_loss
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optimizer.state_dict(),
                'sched_state_dict': scheduler.state_dict(),
                'loss': best_loss,
                'epoch': epoch,
                'metrics': metrics
            }
           
            checkpoint_path = f"{output_path}/best_ckpt_epoch={epoch}.ckpt"
            torch.save(state_dict, checkpoint_path)
            print(f"Best loss: {round(best_loss,5)}")
            print("*"*10 + "Current best checkpoint is saved at " + checkpoint_path + "*"*10)
            
            # Plot confusion matrix for best model
            report = plot_confusion_matrix(all_labels, all_predictions, output_path, 
                                          label_map=train_dataset.label_map)
            
            # Save classification report
            with open(f"{output_path}/best_classification_report.json", 'w') as f:
                json.dump(report, f, indent=4)
    
    # Calculate training time
    training_time = time.time() - start_time
    print(f"Total training time: {training_time/60:.2f} minutes")
    
    # Plot training progress
    plot_training_progress(train_losses_history, valid_losses_history, metrics_history, output_path)
    
    # Plot learning rate
    plot_learning_rate(learning_rates, output_path)
    
    # Save training history for future reference
    history = {
        'train_losses': train_losses_history,
        'valid_losses': valid_losses_history,
        'metrics': metrics_history,
        'training_time': training_time,
        'best_epoch': epoch,
        'best_loss': best_loss
    }
    
    with open(f"{output_path}/training_history.json", 'w') as f:
        json.dump(history, f, indent=4)
    
    return model, tokenizer

# Function to generate predictions from summarization model
def generate_summaries(model, tokenizer, test_data, output_path, batch_size=4, num_beams=4, max_length=150):
    """
    Generate summaries from the trained summarization model.
    """
    model.eval()
    test_dataset = CustomDataset(test_data, tokenizer)
    test_dataloader = test_create_dataloader(test_dataset, batch_size)
    
    all_generated_summaries = []
    all_reference_summaries = []
    all_perspectives = []
    
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Generate summaries
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True
            )
            
            # Decode the generated summaries
            generated_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            # Get reference summaries and perspectives
            reference_summaries = [tokenizer.decode(batch['labels'][i], skip_special_tokens=True) 
                                 for i in range(len(batch['labels']))]
            perspectives = batch['perspective']
            
            # Append to lists
            all_generated_summaries.extend(generated_summaries)
            all_reference_summaries.extend(reference_summaries)
            all_perspectives.extend(perspectives)
    
    # Calculate ROUGE scores
    rouge_results = metric.compute(predictions=all_generated_summaries, references=all_reference_summaries)
    
    # Calculate BLEU scores
    smoothie = SmoothingFunction().method1
    bleu_scores = []
    for gen, ref in zip(all_generated_summaries, all_reference_summaries):
        gen_tokens = nltk.word_tokenize(gen.lower())
        ref_tokens = [nltk.word_tokenize(ref.lower())]
        bleu_score = sentence_bleu(ref_tokens, gen_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu_score)
    
    # Save results
    results = {
        'generated': all_generated_summaries,
        'reference': all_reference_summaries,
        'perspective': all_perspectives,
        'rouge': rouge_results,
        'bleu': np.mean(bleu_scores)
    }
    
    with open(f"{output_path}/summary_results.json", 'w') as f:
        json.dump(results, f, indent=4)
    
    # Plot ROUGE scores by perspective
    perspectives_set = list(set(all_perspectives))
    rouge_by_perspective = {p: {'rouge1': [], 'rouge2': [], 'rougeL': []} for p in perspectives_set}
    
    # Group ROUGE scores by perspective
    for i, p in enumerate(all_perspectives):
        rouge1 = metric.compute(predictions=[all_generated_summaries[i]], 
                              references=[all_reference_summaries[i]])['rouge1']
        rouge2 = metric.compute(predictions=[all_generated_summaries[i]], 
                              references=[all_reference_summaries[i]])['rouge2']
        rougeL = metric.compute(predictions=[all_generated_summaries[i]], 
                              references=[all_reference_summaries[i]])['rougeL']
        
        rouge_by_perspective[p]['rouge1'].append(rouge1)
        rouge_by_perspective[p]['rouge2'].append(rouge2)
        rouge_by_perspective[p]['rougeL'].append(rougeL)
    
    # Calculate mean ROUGE scores by perspective
    rouge_means = {p: {k: np.mean(v) for k, v in scores.items()} 
                 for p, scores in rouge_by_perspective.items()}
    
    # Plot
    plt.figure(figsize=(12, 6))
    x = np.arange(len(perspectives_set))
    width = 0.25
    
    plt.bar(x - width, [rouge_means[p]['rouge1'] for p in perspectives_set], width, label='ROUGE-1')
    plt.bar(x, [rouge_means[p]['rouge2'] for p in perspectives_set], width, label='ROUGE-2')
    plt.bar(x + width, [rouge_means[p]['rougeL'] for p in perspectives_set], width, label='ROUGE-L')
    
    plt.xlabel('Perspective')
    plt.ylabel('Score')
    plt.title('ROUGE Scores by Perspective')
    plt.xticks(x, perspectives_set)
    plt.legend()
    plt.savefig(f"{output_path}/rouge_by_perspective.png")
    plt.close()
    
    return results

# Function to train summarization model
def train_seq2seq_model(train_data_path, valid_data_path, output_path, 
                        model_name="t5-base", train_batch_size=4, valid_batch_size=4,
                        learning_rate=5e-5, num_epochs=3):
    """
    Train a sequence-to-sequence model for text summarization.
    """
    # Load data
    with open(train_data_path, 'r') as json_file:
        train_data = json.load(json_file)
    with open(valid_data_path, 'r') as json_file:
        valid_data = json.load(json_file)
    
    # Initialize tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    
    # Create datasets
    train_dataset = CustomDataset(train_data, tokenizer)
    valid_dataset = CustomDataset(valid_data, tokenizer)
    
    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    
    # Define training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=f"{output_path}",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=valid_batch_size,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        save_strategy="epoch",
        logging_dir=f"{output_path}/logs",
        logging_steps=100,
    )
    
    # Initialize Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    # Train model
    start_time = time.time()
    print("Starting training...")
    trainer.train()
    training_time = time.time() - start_time
    print(f"Training completed in {training_time/60:.2f} minutes")
    
    # Save the final model
    trainer.save_model(f"{output_path}/final_model")
    
    return model, tokenizer



Using device: cuda


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# # Main function to run the entire pipeline
# def main():
#     """
#     Run the entire pipeline: training, evaluation, and visualization.
#     """
#     parser = argparse.ArgumentParser(description='Train and evaluate NLP models for perspective classification and summarization')
    
#     # Add arguments
#     parser.add_argument('--train_file', type=str, default=f'{BASE_PATH}/train.json', 
#                         help='Path to training data file')
#     parser.add_argument('--valid_file', type=str, default=f'{BASE_PATH}/valid.json', 
#                         help='Path to validation data file')
#     parser.add_argument('--test_file', type=str, default=f'{BASE_PATH}/test.json', 
#                         help='Path to test data file')
#     parser.add_argument('--output_dir', type=str, default=OUTPUT_PATH, 
#                         help='Directory to save outputs')
#     parser.add_argument('--task', type=str, choices=['classifier', 'summarizer', 'all'], default='all',
#                         help='Task to run: classifier, summarizer, or all')
#     parser.add_argument('--classifier_epochs', type=int, default=5, 
#                         help='Number of epochs for classifier training')
#     parser.add_argument('--summarizer_epochs', type=int, default=3, 
#                         help='Number of epochs for summarizer training')
#     parser.add_argument('--classifier_batch_size', type=int, default=8, 
#                         help='Batch size for classifier training')
#     parser.add_argument('--summarizer_batch_size', type=int, default=4, 
#                         help='Batch size for summarizer training')
#     parser.add_argument('--classifier_lr', type=float, default=1e-5, 
#                         help='Learning rate for classifier')
#     parser.add_argument('--summarizer_lr', type=float, default=5e-5, 
#                         help='Learning rate for summarizer')
#     parser.add_argument('--summarizer_model', type=str, default='t5-base', 
#                         help='Model name for summarizer')
    
#     args = parser.parse_args()
    
#     # Set up logging
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     log_file = f"{args.output_dir}/run_log_{timestamp}.txt"
    
#     # Print and log basic info
#     print(f"Starting experiment at {timestamp}")
#     print(f"Using device: {device}")
#     print(f"Args: {args}")
    
#     # Training and evaluation
#     if args.task in ['classifier', 'all']:
#         print("\n" + "="*50)
#         print("TRAINING CLASSIFIER MODEL")
#         print("="*50)
        
#         # Train classifier
#         classifier_model, classifier_tokenizer = train_classifier(
#             train_data_path=args.train_file,
#             valid_data_path=args.valid_file,
#             output_path=args.output_dir,
#             train_batch_size=args.classifier_batch_size,
#             valid_batch_size=args.classifier_batch_size // 2,
#             learning_rate=args.classifier_lr,
#             warmup_steps=100,
#             epochs=args.classifier_epochs
#         )
        
#         # Load test data for classifier evaluation
#         with open(args.test_file, 'r') as f:
#             test_data = json.load(f)
        
#         # Create test dataset and dataloader
#         test_dataset = ClassifierCustomDataset(test_data, classifier_tokenizer)
#         test_dataloader = test_create_dataloader(test_dataset, args.classifier_batch_size)
        
#         # Evaluate on test data
#         print("\nEvaluating classifier on test data...")
#         test_results = evaluate_classifier_test(test_dataloader, classifier_model, classifier_tokenizer)
        
#         # Visualize sample predictions
#         visualize_sample_predictions(test_results, num_samples=5, output_path=args.output_dir)
        
#         # Plot distribution of predictions
#         plot_prediction_distribution(test_results['predictions'], args.output_dir)
        
#         print("\nClassifier training and evaluation completed!")
    
#     if args.task in ['summarizer', 'all']:
#         print("\n" + "="*50)
#         print("TRAINING SUMMARIZATION MODEL")
#         print("="*50)
        
#         # Train summarization model
#         summarizer_model, summarizer_tokenizer = train_seq2seq_model(
#             train_data_path=args.train_file,
#             valid_data_path=args.valid_file,
#             output_path=args.output_dir,
#             model_name=args.summarizer_model,
#             train_batch_size=args.summarizer_batch_size,
#             valid_batch_size=args.summarizer_batch_size // 2,
#             learning_rate=args.summarizer_lr,
#             num_epochs=args.summarizer_epochs
#         )
        
#         # Load test data for summarizer evaluation
#         with open(args.test_file, 'r') as f:
#             test_data = json.load(f)
        
#         # Generate summaries and evaluate
#         print("\nGenerating summaries on test data...")
#         summary_results = generate_summaries(
#             model=summarizer_model,
#             tokenizer=summarizer_tokenizer,
#             test_data=test_data,
#             output_path=args.output_dir,
#             batch_size=args.summarizer_batch_size,
#             num_beams=4,
#             max_length=150
#         )
        
#         # Print summary evaluation results
#         print(f"\nSummarization Results:")
#         print(f"ROUGE-1: {summary_results['rouge']['rouge1']:.4f}")
#         print(f"ROUGE-2: {summary_results['rouge']['rouge2']:.4f}")
#         print(f"ROUGE-L: {summary_results['rouge']['rougeL']:.4f}")
#         print(f"BLEU: {summary_results['bleu']:.4f}")
        
#         print("\nSummarizer training and evaluation completed!")
    
#     # Generate combined report
#     if args.task == 'all':
#         generate_combined_report(args.output_dir)
    
#     print("\n" + "="*50)
#     print(f"EXPERIMENT COMPLETED! Results saved to {args.output_dir}")
#     print("="*50)

# def generate_combined_report(output_path):
#     """
#     Generate a combined report of classifier and summarizer results.
#     """
#     # Create a comprehensive report figure
#     plt.figure(figsize=(15, 12))
    
#     # Add classifier confusion matrix
#     try:
#         cm_img = plt.imread(f"{output_path}/plots/confusion_matrix.png")
#         plt.subplot(2, 2, 1)
#         plt.imshow(cm_img)
#         plt.axis('off')
#         plt.title("Classifier Confusion Matrix")
#     except:
#         print("Could not find confusion matrix image")
    
#     # Add training progress
#     try:
#         prog_img = plt.imread(f"{output_path}/plots/training_progress.png")
#         plt.subplot(2, 2, 2)
#         plt.imshow(prog_img)
#         plt.axis('off')
#         plt.title("Training Progress")
#     except:
#         print("Could not find training progress image")
    
#     # Add summarizer ROUGE scores
#     try:
#         rouge_img = plt.imread(f"{output_path}/plots/rouge_by_perspective.png")
#         plt.subplot(2, 2, 3)
#         plt.imshow(rouge_img)
#         plt.axis('off')
#         plt.title("Summarizer ROUGE Scores by Perspective")
#     except:
#         print("Could not find ROUGE scores image")
    
#     # Add prediction distribution
#     try:
#         dist_img = plt.imread(f"{output_path}/plots/prediction_distribution.png")
#         plt.subplot(2, 2, 4)
#         plt.imshow(dist_img)
#         plt.axis('off')
#         plt.title("Prediction Distribution")
#     except:
#         print("Could not find prediction distribution image")
    
#     plt.tight_layout()
#     plt.savefig(f"{output_path}/combined_report.png", dpi=300)
#     plt.close()
    
#     # Gather metrics from files
#     metrics = {}
#     try:
#         with open(f"{output_path}/training_history.json", 'r') as f:
#             training_history = json.load(f)
#         metrics['training'] = training_history
#     except:
#         print("Could not load training history")
    
#     try:
#         with open(f"{output_path}/best_classification_report.json", 'r') as f:
#             class_report = json.load(f)
#         metrics['classification'] = class_report
#     except:
#         print("Could not load classification report")
    
#     try:
#         with open(f"{output_path}/generated/summary_results.json", 'r') as f:
#             summary_results = json.load(f)
#         metrics['summarization'] = {
#             'rouge': summary_results['rouge'],
#             'bleu': summary_results['bleu']
#         }
#     except:
#         print("Could not load summarization results")
    
#     # Save combined metrics
#     with open(f"{output_path}/combined_metrics.json", 'w') as f:
#         json.dump(metrics, f, indent=4)
    
#     # Generate simple text report
#     with open(f"{output_path}/final_report.txt", 'w') as f:
#         f.write("=" * 50 + "\n")
#         f.write("COMBINED RESULTS REPORT\n")
#         f.write("=" * 50 + "\n\n")
        
#         # Classifier results
#         f.write("CLASSIFIER RESULTS:\n")
#         f.write("-" * 30 + "\n")
#         if 'classification' in metrics:
#             f.write(f"Accuracy: {metrics['classification']['accuracy']:.4f}\n")
#             f.write("Class-wise Performance:\n")
#             for cls in metrics['classification']:
#                 if cls not in ['accuracy', 'macro avg', 'weighted avg']:
#                     f.write(f"  {cls}: F1={metrics['classification'][cls]['f1']:.4f}, "
#                            f"Precision={metrics['classification'][cls]['precision']:.4f}, "
#                            f"Recall={metrics['classification'][cls]['recall']:.4f}\n")
#         else:
#             f.write("No classification metrics available\n")
        
#         f.write("\n")
        
#         # Summarizer results
#         f.write("SUMMARIZER RESULTS:\n")
#         f.write("-" * 30 + "\n")
#         if 'summarization' in metrics:
#             f.write(f"ROUGE-1: {metrics['summarization']['rouge']['rouge1']:.4f}\n")
#             f.write(f"ROUGE-2: {metrics['summarization']['rouge']['rouge2']:.4f}\n")
#             f.write(f"ROUGE-L: {metrics['summarization']['rouge']['rougeL']:.4f}\n")
#             f.write(f"BLEU: {metrics['summarization']['bleu']:.4f}\n")
#         else:
#             f.write("No summarization metrics available\n")
        
#         f.write("\n")
#         f.write("=" * 50 + "\n")
#         f.write("END OF REPORT\n")
#         f.write("=" * 50 + "\n")

# # Run the main function if script is executed directly
# if __name__ == "__main__":
#     main()

usage: colab_kernel_launcher.py [-h] [--train_file TRAIN_FILE] [--valid_file VALID_FILE]
                                [--test_file TEST_FILE] [--output_dir OUTPUT_DIR]
                                [--task {classifier,summarizer,all}]
                                [--classifier_epochs CLASSIFIER_EPOCHS]
                                [--summarizer_epochs SUMMARIZER_EPOCHS]
                                [--classifier_batch_size CLASSIFIER_BATCH_SIZE]
                                [--summarizer_batch_size SUMMARIZER_BATCH_SIZE]
                                [--classifier_lr CLASSIFIER_LR] [--summarizer_lr SUMMARIZER_LR]
                                [--summarizer_model SUMMARIZER_MODEL]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-935c0797-664f-40b0-aba1-ecb0bf1b7e3c.json


SystemExit: 2

In [15]:
# Main function adapted for Jupyter Notebook
def main(train_file=f'{BASE_PATH}/train.json',
         valid_file=f'{BASE_PATH}/valid.json',
         test_file=f'{BASE_PATH}/test.json',
         output_dir=OUTPUT_PATH,
         task='all',
         classifier_epochs=5,
         summarizer_epochs=3,
         classifier_batch_size=8,
         summarizer_batch_size=4,
         classifier_lr=1e-5,
         summarizer_lr=5e-5,
         summarizer_model='t5-base'):
    """
    Run the entire pipeline: training, evaluation, and visualization.
    All parameters are optional with defaults.
    
    Parameters:
    -----------
    train_file : str
        Path to training data file
    valid_file : str
        Path to validation data file
    test_file : str
        Path to test data file
    output_dir : str
        Directory to save outputs
    task : str
        Task to run: 'classifier', 'summarizer', or 'all'
    classifier_epochs : int
        Number of epochs for classifier training
    summarizer_epochs : int
        Number of epochs for summarizer training
    classifier_batch_size : int
        Batch size for classifier training
    summarizer_batch_size : int
        Batch size for summarizer training
    classifier_lr : float
        Learning rate for classifier
    summarizer_lr : float
        Learning rate for summarizer
    summarizer_model : str
        Model name for summarizer
    """
    # Set up logging
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"{output_dir}/run_log_{timestamp}.txt"
    
    # Print basic info
    print(f"Starting experiment at {timestamp}")
    print(f"Using device: {device}")
    
    # Training and evaluation
    if task in ['classifier', 'all']:
        print("\n" + "="*50)
        print("TRAINING CLASSIFIER MODEL")
        print("="*50)
        
        # Train classifier
        classifier_model, classifier_tokenizer = train_classifier(
            train_data_path=train_file,
            valid_data_path=valid_file,
            output_path=output_dir,
            train_batch_size=classifier_batch_size,
            valid_batch_size=classifier_batch_size // 2,
            learning_rate=classifier_lr,
            warmup_steps=100,
            epochs=classifier_epochs
        )
        
        # Load test data for classifier evaluation
        with open(test_file, 'r') as f:
            test_data = json.load(f)
        
        # Create test dataset and dataloader
        test_dataset = ClassifierCustomDataset(test_data, classifier_tokenizer)
        test_dataloader = test_create_dataloader(test_dataset, classifier_batch_size)
        
        # Evaluate on test data
        print("\nEvaluating classifier on test data...")
        test_results = evaluate_classifier_test(test_dataloader, classifier_model, classifier_tokenizer)
        
        # Visualize sample predictions
        visualize_sample_predictions(test_results, num_samples=5, output_path=output_dir)
        
        # Plot distribution of predictions
        plot_prediction_distribution(test_results['predictions'], output_dir)
        
        print("\nClassifier training and evaluation completed!")
    
    if task in ['summarizer', 'all']:
        print("\n" + "="*50)
        print("TRAINING SUMMARIZATION MODEL")
        print("="*50)
        
        # Train summarization model
        summarizer_model, summarizer_tokenizer = train_seq2seq_model(
            train_data_path=train_file,
            valid_data_path=valid_file,
            output_path=output_dir,
            model_name=summarizer_model,
            train_batch_size=summarizer_batch_size,
            valid_batch_size=summarizer_batch_size // 2,
            learning_rate=summarizer_lr,
            num_epochs=summarizer_epochs
        )
        
        # Load test data for summarizer evaluation
        with open(test_file, 'r') as f:
            test_data = json.load(f)
        
        # Generate summaries and evaluate
        print("\nGenerating summaries on test data...")
        summary_results = generate_summaries(
            model=summarizer_model,
            tokenizer=summarizer_tokenizer,
            test_data=test_data,
            output_path=output_dir,
            batch_size=summarizer_batch_size,
            num_beams=4,
            max_length=150
        )
        
        # Print summary evaluation results
        print(f"\nSummarization Results:")
        print(f"ROUGE-1: {summary_results['rouge']['rouge1']:.4f}")
        print(f"ROUGE-2: {summary_results['rouge']['rouge2']:.4f}")
        print(f"ROUGE-L: {summary_results['rouge']['rougeL']:.4f}")
        print(f"BLEU: {summary_results['bleu']:.4f}")
        
        print("\nSummarizer training and evaluation completed!")
    
    # Generate combined report
    if task == 'all':
        generate_combined_report(output_dir)
    
    print("\n" + "="*50)
    print(f"EXPERIMENT COMPLETED! Results saved to {output_dir}")
    print("="*50)
    
    return {
        'classifier_model': classifier_model if task in ['classifier', 'all'] else None,
        'classifier_tokenizer': classifier_tokenizer if task in ['classifier', 'all'] else None,
        'summarizer_model': summarizer_model if task in ['summarizer', 'all'] else None,
        'summarizer_tokenizer': summarizer_tokenizer if task in ['summarizer', 'all'] else None
    }

# Example usage in Jupyter Notebook:
models = main(
    task='all',                      # 'classifier', 'summarizer', or 'all'
    classifier_epochs=3,             # Reduce for faster execution
    summarizer_epochs=2,             # Reduce for faster execution
    classifier_batch_size=32,         # Adjust based on your GPU memory
    summarizer_batch_size=16          # Adjust based on your GPU memory
)

Starting experiment at 20250415_180255
Using device: cuda

TRAINING CLASSIFIER MODEL
Initial best_loss: inf


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


################################################## Epoch: 1 ##################################################


  1%|▏         | 1/70 [00:02<02:28,  2.15s/it]

Epoch 1, Batch 0/70, Loss: 1.6334


...

100%|██████████| 70/70 [02:27<00:00,  2.11s/it]


Train loss: 0.9678604668272394 for epoch: 1
Validation processing...


100%|██████████| 60/60 [00:14<00:00,  4.07it/s]


Validation metrics for epoch 1:
  loss: 0.0265
  accuracy: 1.0000
  precision: 1.0000
  recall: 1.0000
  f1: 1.0000
Best loss: 0.02647
**********Current best checkpoint is saved at /kaggle/working//best_ckpt_epoch=1.ckpt**********


ValueError: Number of classes, 1, does not match size of target_names, 5. Try specifying the labels parameter

In [ ]:
# Interactive widgets implementation
# First make sure to install ipywidgets if not already installed
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display

def run_with_widgets():
    # Create widgets for each parameter
    task_widget = widgets.RadioButtons(
        options=['classifier', 'summarizer', 'all'],
        value='all',
        description='Task:',
        disabled=False
    )
    
    classifier_epochs_widget = widgets.IntSlider(
        value=3,
        min=1,
        max=10,
        step=1,
        description='Classifier Epochs:',
        disabled=False,
        continuous_update=False
    )
    
    summarizer_epochs_widget = widgets.IntSlider(
        value=2,
        min=1,
        max=5,
        step=1,
        description='Summarizer Epochs:',
        disabled=False,
        continuous_update=False
    )
    
    classifier_batch_size_widget = widgets.IntSlider(
        value=8,
        min=2,
        max=16,
        step=2,
        description='Classifier Batch Size:',
        disabled=False,
        continuous_update=False
    )
    
    summarizer_batch_size_widget = widgets.IntSlider(
        value=4,
        min=2,
        max=8,
        step=2,
        description='Summarizer Batch Size:',
        disabled=False,
        continuous_update=False
    )
    
    classifier_lr_widget = widgets.FloatLogSlider(
        value=1e-5,
        base=10,
        min=-6,  # 10^-6
        max=-4,  # 10^-4
        step=0.1,
        description='Classifier LR:',
        disabled=False,
        continuous_update=False
    )
    
    summarizer_lr_widget = widgets.FloatLogSlider(
        value=5e-5,
        base=10,
        min=-6,  # 10^-6
        max=-4,  # 10^-4
        step=0.1,
        description='Summarizer LR:',
        disabled=False,
        continuous_update=False
    )
    
    model_widget = widgets.Dropdown(
        options=['t5-small', 't5-base', 't5-large'],
        value='t5-base',
        description='Summarizer Model:',
        disabled=False
    )
    
    output_widget = widgets.Output()
    
    # Button to start training
    button = widgets.Button(
        description='Start Training',
        disabled=False,
        button_style='success',
        tooltip='Click to start the training process',
        icon='play'
    )
    
    # Function to run when button is clicked
    def on_button_clicked(b):
        with output_widget:
            # Clear previous outputs
            output_widget.clear_output()
            
            # Run the main function with the selected parameters
            print("Starting training with selected parameters...")
            models = main(
                task=task_widget.value,
                classifier_epochs=classifier_epochs_widget.value,
                summarizer_epochs=summarizer_epochs_widget.value,
                classifier_batch_size=classifier_batch_size_widget.value,
                summarizer_batch_size=summarizer_batch_size_widget.value,
                classifier_lr=classifier_lr_widget.value,
                summarizer_lr=summarizer_lr_widget.value,
                summarizer_model=model_widget.value
            )
            print("Training complete!")
            
    # Attach the function to the button
    button.on_click(on_button_clicked)
    
    # Create layout for the widgets
    left_box = widgets.VBox([task_widget, classifier_epochs_widget, summarizer_epochs_widget, 
                            classifier_batch_size_widget, summarizer_batch_size_widget])
    right_box = widgets.VBox([classifier_lr_widget, summarizer_lr_widget, model_widget, button])
    controls = widgets.HBox([left_box, right_box])
    
    # Display the widgets and output
    display(controls)
    display(output_widget)

# Run this cell to display the interactive interface
run_with_widgets()